In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import torch

In [ ]:
train_data = pd.read_csv('train.csv')
print(train_data)
print(train_data.shape)

In [ ]:
user_tag = pd.read_csv('user_tags.csv')
job_tag = pd.read_csv('job_tags.csv')
raw_tags = pd.read_csv('tags.csv')

In [ ]:
print(user_tag.sample(5))

In [ ]:
print(train_data.loc[0])

In [ ]:
#user1 = train_data["userID"] == "fe292163d06253b716e9a0099b42031d"
#print(train_data[user1])    # fe292163d06253b716e9a0099b42031d라는 유저는 27개의 지원 데이터가 있음

In [ ]:
users = user_tag["userID"].unique()
print(users.size)    # 196명의 사용자 대상 6000개의 데이터 확인

In [ ]:
tags = raw_tags["tagID"]

In [ ]:
print(len(raw_tags['keyword'].unique()))
print(len(raw_tags['tagID'].unique()))

In [ ]:
users_mat = np.zeros(shape=(users.size, tags.size))    # 각각의 유저들이 가진 태그들을 0,1로 이루어진 vector로 표현한 행렬
print(users_mat.shape)
for userID, tagID in user_tag.values:
    users_mat[np.where(users == userID)[0][0]][np.where(tags == tagID)[0][0]] = 1
print(np.sum(users_mat[103]))
print(np.sum(users_mat[150]))
print(np.sum(users_mat[174]))

In [ ]:
jobs = job_tag["jobID"].unique()
print(jobs.size)

In [ ]:
jobs_mat = np.zeros(shape=(jobs.size, tags.size))
print(jobs_mat.shape)
for jobID, tagID in job_tag.values:
    jobs_mat[np.where(jobs == jobID)[0][0]][np.where(tags == tagID)[0][0]] = 1
print(np.sum(jobs_mat[0]))

In [ ]:
print(train_data.applied.sum())
print(len(train_data.applied))

In [ ]:
#print(train_data[['userID',"jobID"]])
testdd = train_data['userID'] + "  " + train_data['jobID']
print(len(pd.unique(testdd)))
#print(len(pd.unique(train_data[['userID',"jobID"]].values.ravel('K'))))

cossim 방식

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
cossim = cosine_similarity
eucdist = euclidean_distances

In [ ]:
bestmatch = -1
user = np.where(users == "fe292163d06253b716e9a0099b42031d")[0][0]
print(user)
for i in range(len(users_mat)):
    if user != i:
        sim = cossim([users_mat[user], users_mat[i]])[0][1]
        #print(sim)
        if bestmatch < sim:
            bestmatch = sim
            bestuser = i
print(bestuser)
print(bestmatch)

In [ ]:
import matplotlib.pyplot as plt
plt.bar(tags, users_mat[103], color='r', linewidth=2)
plt.show()
plt.bar(tags, users_mat[150], color='g', linewidth=2)
plt.show()

In [ ]:
import seaborn as sns
simmat = np.ndarray(shape=(len(users), len(users)))
print(simmat.shape)
for i in range(len(users)):
    for j in range(len(users)):
        simmat[i][j] = cossim([users_mat[i], users_mat[j]])[0][1]

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.heatmap(simmat, ax=ax)


In [ ]:
#plt.imshow(simmat, cmap='hot', interpolation='nearest')
#plt.show()

In [ ]:
print(raw_tags[users_mat[103] == 1].keyword)
print(raw_tags[users_mat[150] == 1].keyword)

In [ ]:
filter = np.logical_and(np.array(users_mat[103] == 1), np.array(users_mat[150] == 1))
print(raw_tags[filter].keyword)

In [ ]:
print(users_mat[np.where(users == 'fe292163d06253b716e9a0099b42031d')[0][0]])

In [ ]:
#print(users_mat[0])
#print(users_mat[1])
#print(jobs_mat[0])
#print(jobs_mat[1])

In [ ]:
def bestuser(user):
    bestmatch = -1
#    user = np.where(users == userID)[0][0]
    for i in range(len(users_mat)):
        if user != i:
            sim = cossim([users_mat[user], users_mat[i]])[0][1]
            #print(sim)
            if bestmatch < sim:
                bestmatch = sim
                bestuser = i
    print(bestuser)
    print(bestmatch)

In [ ]:
def bestjob(user):
    bestmatch = -1
#    user = np.where(users == userID)[0][0]
    for i in range(len(users_mat)):
        sim = cossim([users_mat[user], jobs_mat[i]])[0][1]
        #print(sim)
        if bestmatch < sim:
            bestmatch = sim
            bestuser = i
    print(bestuser)
    print(bestmatch)

In [ ]:
def cotag_user(user1, user2):
    print('user1: ', raw_tags[users_mat[user1] == 1].keyword)
    print('user2: ', raw_tags[users_mat[user2] == 1].keyword)
    filter = np.logical_and(np.array(users_mat[user1] == 1), np.array(users_mat[user2] == 1))
    print(raw_tags[filter].keyword)

In [ ]:
def cotag_job(user, job):
    print('user1: ', raw_tags[users_mat[user] == 1].keyword)
    print('user2: ', raw_tags[jobs_mat[job] == 1].keyword)
    filter = np.logical_and(np.array(users_mat[user] == 1), np.array(jobs_mat[job] == 1))
    print(raw_tags[filter].keyword)

In [ ]:
print(bestjob(22))
print(cotag_job(22, 37))

In [ ]:
"""
class Net(torch.nn.module):
    def __init__(self):
        super.__init__(self)
        
    def forward(self, x):
"""     

gmf

In [ ]:
#user가 job에 지원했는지 표현하는 행렬(했으면 2, 아니면 1, data가 없으면 0)
user_jobmat = np.full(shape=(len(users),len(jobs)), fill_value=0)
for idx in train_data[:5000].values:
    uid = idx[0]
    jid = idx[1]
    app = idx[2]+1
    user_jobmat[np.where(users == uid)[0][0]][np.where(jobs == jid)[0][0]] = app
#print(user_jobmat)
#print(user_jobmat.shape)
#print(user_jobmat[1])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NCF(nn.Module):
    def __init__(self, n_user=196, n_job=733, n_feature=100, d_out=2, dp=0.02):
        super().__init__()
        self.relu = nn.ReLU()
        self.user_embed = nn.Linear(n_job, n_feature)
        self.job_embed = nn.Linear(n_user, n_feature)
        
        self.ncfLayer = nn.Sequential(
#            nn.Linear(n_feature*2, n_feature),
#            nn.ReLU(),
#            nn.Dropout(dp),
#            nn.BatchNorm1d(n_feature),
            nn.Linear(n_feature, 50),
            nn.ReLU(),
            nn.Dropout(dp),
            nn.BatchNorm1d(50),
            nn.Linear(50, d_out),
            nn.Linear(d_out, 1)    # BCELOSS적용시
        )
        
    def forward(self, user, job):
        user = self.user_embed(user)
        job = self.job_embed(job)
#        x = torch.cat((user, job), dim=1)
        x = torch.mul(user, job)
        x = self.ncfLayer(x)
        return x

In [ ]:
criterion = nn.BCEWithLogitsLoss()
model = NCF()
optim = torch.optim.Adam(model.parameters(), lr=0.006, weight_decay=1e-8)

데이터셋, 샘플러

In [ ]:
class UJDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        super().__init__()
#        self.user_jobmat = np.full(shape=(len(users),len(jobs)), fill_value=-1)
#        for idx in data:
#            uid = idx[0]
#            jid = idx[1]
#            app = idx[2]
#            self.user_jobmat[np.where(users == uid)[0][0]][np.where(jobs == jid)[0][0]] = app
        self.sample = np.ndarray(shape=(len(data), 3), dtype=int)
        for i in range(len(data)):
            self.sample[i][0] = np.where(users == data.iloc[i][0])[0][0]
            self.sample[i][1] = np.where(jobs == data.iloc[i][1])[0][0]
            self.sample[i][2] = data.iloc[i][2]
    def __len__(self):
        return len(self.sample)

    def __getitem__(self, i):
        return user_jobmat[self.sample[i][0],:], user_jobmat[:,self.sample[i][1]].T, self.sample[i][2]

In [ ]:
train_train_data = UJDataset(train_data[:5000])
train_test_data = UJDataset(train_data[5000:6000])

In [ ]:
targets = train_data[:5000].applied
class_sample_count = np.array([len(np.where(targets==t)[0]) for t in np.unique(targets)])
weights = 1. / class_sample_count
samples_weights = weights[targets]
trainsampler = torch.utils.data.sampler.WeightedRandomSampler(samples_weights, len(samples_weights), replacement=True)

In [ ]:
targets = train_data[5000:6000].applied
class_sample_count = np.array([len(np.where(targets==t)[0]) for t in np.unique(targets)])
weights = 1. / class_sample_count
samples_weights = weights[targets]
testsampler = torch.utils.data.sampler.WeightedRandomSampler(samples_weights, len(samples_weights), replacement=True)

In [ ]:
#샘플러 사용
trainloader = torch.utils.data.DataLoader(train_train_data, batch_size=5,sampler=trainsampler)
testloader = torch.utils.data.DataLoader(train_test_data, batch_size=5, sampler=testsampler)
#print(next(iter(trainloader)))

In [ ]:
rfefws = next(iter(trainloader))
#print(rfefws[0].shape)

In [ ]:
#랜덤
#trainloader = torch.utils.data.DataLoader(train_train_data, batch_size=5,shuffle=True)
#testloader = torch.utils.data.DataLoader(train_test_data, batch_size=5, shuffle=True)
#print(next(iter(trainloader)))

In [ ]:
import time

훈련 시작

In [ ]:
epoch = 10
train_loss = 0
all_losses = []
for eps in range(epoch):
    print('training', eps+1 , 'epoch')
    for i, data in enumerate(trainloader, 0):
        model.train()
        uid, jid, labels = data
#        uid = data[:,0]
#        jid = data[:,1]
#        labels = data[:,2]
#        uid = user_jobmat[uid,:]
#        jid = user_jobmat[:,jid]
#        labels = torch.tensor(labels, dtype=torch.long)
        optim.zero_grad()
#        input_user = users_mat[np.where(users == train_train_data.iloc[i][0])[0][0]]
#        input_job = jobs_mat[np.where(jobs == train_train_data.iloc[i][1])[0][0]]                    
#        input_data = torch.tensor([users_mat[np.where(users == train_train_data.iloc[i][0])[0][0]]])
#        label = torch.zeros(2)    # 0 -> [1,0], 1 -> [0, 1]
#        label[train_train_data.iloc[i][2]] = 1
#        label = train_train_data.iloc[i][2]
        answers = model(uid.float(), jid.float())
        labels =labels.unsqueeze(1).float()
        loss = criterion(answers, labels)
        loss.backward();
        optim.step()
        train_loss += loss.item()
#        print(answers)
#        print(labels)

        if i % 100 == 99:    # print every 100 mini-batches
#            print(answers)
#            print(labels)
#            print("a: ", answers.T, " l: ", labels.T)
            
#            _, predicted = torch.max(answers.data, 1)
#            print("p: ", predicted, " l: ", labels.T)
#            time.sleep(1)
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, train_loss / 100))
            all_losses.append(train_loss)
            train_loss = 0.0


In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(len(all_losses)), all_losses)
plt.show()

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        uid, jid, labels = data
        labels = torch.tensor(labels, dtype=torch.long)
        outputs = model(uid.float(), jid.float())
#        print("o: ", outputs)
        _, predicted = torch.max(outputs.data, 1)
#        print("p: ", predicted)
#        print("l: ", labels)
#        print("a: ", outputs.T, " p: ",  predicted, " l: ", labels)
#        time.sleep(3)
#        total += labels.sum()
        total += labels.size(0)
#        correct += labels[predicted == labels].sum()
        correct += (predicted == labels).sum().item()

print('Accuracy: %d %%' % (
    100 * correct / total))

tag 정보 반영

In [ ]:
class UJDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        super().__init__()
#        self.user_jobmat = np.full(shape=(len(users),len(jobs)), fill_value=-1)
#        for idx in data:
#            uid = idx[0]
#            jid = idx[1]
#            app = idx[2]
#            self.user_jobmat[np.where(users == uid)[0][0]][np.where(jobs == jid)[0][0]] = app
        self.sample = np.ndarray(shape=(len(data), 3), dtype=int)
        for i in range(len(data)):
            self.sample[i][0] = np.where(users == data.iloc[i][0])[0][0]
            self.sample[i][1] = np.where(jobs == data.iloc[i][1])[0][0]
            self.sample[i][2] = data.iloc[i][2]
    def __len__(self):
        return len(self.sample)

    def __getitem__(self, i):
        return self.sample[i]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NCFwithTag(nn.Module):
    def __init__(self, n_user=196, n_job=733, n_feature=100, d_out=2, dp=0.02):
        super().__init__()
        n_tags = 887; d_embed = 400; d_hidden = 200
        self.relu = nn.ReLU()
        self.user_embed = nn.Linear(n_job, n_feature)
        self.job_embed = nn.Linear(n_user, n_feature)
        
        self.ncfLayer = nn.Sequential(
#            nn.Linear(n_feature*2, n_feature),
#            nn.ReLU(),
#            nn.Dropout(dp),
#            nn.BatchNorm1d(n_feature),
            nn.Linear(n_feature, 50),
            nn.ReLU(),
            nn.Dropout(dp),
            nn.BatchNorm1d(50),
            nn.Linear(50, d_out),
            nn.Linear(d_out, 1)    # BCELOSS적용시
        )
        self.module1 = nn.Sequential(    # usertags embedding
        nn.Linear(n_tags, d_embed),
        nn.ReLU(),
        nn.Dropout(dp),
        nn.BatchNorm1d(d_embed),
        nn.Linear(d_embed, d_hidden),
        nn.ReLU(),
        nn.Dropout(dp),
        nn.BatchNorm1d(d_hidden),
        nn.Linear(d_hidden, n_feature),
        nn.ReLU(),
        nn.BatchNorm1d(n_feature)
        )
        
        # self.module2 = nn.Sequential(    # jobtags embedding
        # nn.Linear(n_tags, d_embed),
        # nn.ELU(),
        # nn.Dropout(dp),
        # nn.BatchNorm1d(d_embed),
        # nn.Linear(d_embed, d_hidden),
        # nn.ELU(),
        # nn.Dropout(dp),
        # nn.BatchNorm1d(d_hidden),
        # nn.Linear(d_hidden, n_feature),
        # nn.ELU(),
        # nn.BatchNorm1d(n_feature)
        # )

    def forward(self, ulv, ut, jlv, jt):
        ulv = self.user_embed(ulv)
        jlv = self.job_embed(jlv)
        ut = self.module1(ut)
        jt = self.module1(jt)
#        x = torch.cat((user, job), dim=1)
        x = torch.mul(ulv, jlv)
        y = torch.mul(ut, jt)
        x = torch.mul(x, y)
        x = self.ncfLayer(x)
        return x

In [ ]:
train_train_data = UJDataset(train_data[:5000])
train_test_data = UJDataset(train_data[5000:6000])
trainloader = torch.utils.data.DataLoader(train_train_data, batch_size=5,sampler=trainsampler)
testloader = torch.utils.data.DataLoader(train_test_data, batch_size=5, sampler=testsampler)
print(next(iter(trainloader)))

In [ ]:
def datahandler(data):
    uid = data[:,0]
    jid = data[:,1]
#    label = data[:,2]
    ulv = user_jobmat[uid,:]    # user latent vector
    jlv = user_jobmat[:,jid].T
    ut = users_mat[uid]
    jt = jobs_mat[jid]
    return ulv, ut, jlv, jt

In [ ]:
criterion = nn.BCEWithLogitsLoss()
model = NCFwithTag()
optim = torch.optim.Adam(model.parameters(), lr=0.006, weight_decay=1e-8)

In [ ]:
epoch = 5
train_loss = 0
all_losses = []
for eps in range(epoch):
    print('training', eps+1 , 'epoch')
    for i, data in enumerate(trainloader, 0):
        model.train()
        labels = data[:,2]
#        uid = data[:,0]
#        jid = data[:,1]
#        labels = data[:,2]
#        uid = user_jobmat[uid,:]
#        jid = user_jobmat[:,jid]
#        labels = torch.tensor(labels, dtype=torch.long)
        optim.zero_grad()
#        input_user = users_mat[np.where(users == train_train_data.iloc[i][0])[0][0]]
#        input_job = jobs_mat[np.where(jobs == train_train_data.iloc[i][1])[0][0]]                    
#        input_data = torch.tensor([users_mat[np.where(users == train_train_data.iloc[i][0])[0][0]]])
#        label = torch.zeros(2)    # 0 -> [1,0], 1 -> [0, 1]
#        label[train_train_data.iloc[i][2]] = 1
#        label = train_train_data.iloc[i][2]
        ulv, ut, jlv, jt = datahandler(data)
        ulv = torch.as_tensor(ulv).float()
        ut = torch.as_tensor(ut).float()
        jlv = torch.as_tensor(jlv).float()
        jt = torch.as_tensor(jt).float()
        answers = model(ulv, ut, jlv, jt)
        labels =labels.unsqueeze(1).float()
        loss = criterion(answers, labels)
        loss.backward();
        optim.step()
        train_loss += loss.item()
#        print(answers)
#        print(labels)

        if i % 100 == 99:    # print every 100 mini-batches
#            print(answers)
#            print(labels)
#            print("a: ", answers.T, " l: ", labels.T)
            
#            _, predicted = torch.max(answers.data, 1)
#            print("p: ", predicted, " l: ", labels.T)
#            time.sleep(1)
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, train_loss / 100))
            all_losses.append(train_loss)
            train_loss = 0.0


In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(len(all_losses)), all_losses)
plt.show()

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        ulv, ut, jlv, jt = datahandler(data)
        ulv = torch.as_tensor(ulv).float()
        ut = torch.as_tensor(ut).float()
        jlv = torch.as_tensor(jlv).float()
        jt = torch.as_tensor(jt).float()
        labels = data[:,2]
        outputs = model(ulv, ut, jlv, jt)
#        print("o: ", outputs)
        _, predicted = torch.max(outputs.data, 1)
#        print("p: ", predicted)
#        print("l: ", labels)
#        print("a: ", outputs.T, " p: ",  predicted, " l: ", labels)
#        time.sleep(3)
#        total += labels.sum()
        total += labels.size(0)
#        correct += labels[predicted == labels].sum()
        correct += (predicted == labels).sum().item()

print('Accuracy: %d %%' % (
    100 * correct / total))

간단한 신경망 생성

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, n_embed=887, d_embed=400, d_hidden=200, d_out=2, dp=0.02):  #887차원의 데이터를 300차원으로 임베딩, 256개 은닉층을 거쳐 10개 데이터 아웃
        super().__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(n_embed, d_embed)
        self.dropout1 = nn.Dropout(dp)
        self.bn1 = nn.BatchNorm1d(d_embed)
        self.fc2 = nn.Linear(d_embed, d_hidden)
        self.dropout2 = nn.Dropout(dp)
        self.bn2 = nn.BatchNorm1d(d_hidden)
        self.fc3 = nn.Linear(d_hidden, d_out)   

    def forward(self, x):
        
        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = self.dropout2(x)
        x = self.relu(x)
        x = self.bn2(x)
        x = self.fc3(x)
        return x

In [ ]:
criterion = nn.CrossEntropyLoss()
model = Net()
optim = torch.optim.Adam(model.parameters(), lr=0.002)

In [ ]:
#train_train_data = train_data[:5000]
#train_test_data = train_data[5000:6000]

In [ ]:
class TagsMatDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        super().__init__()
        self.samples = np.ndarray(shape=(len(data), len(tags)))
        self.labels = np.zeros(shape=(len(data)))
        for i in range(len(data)):
            self.samples[i] = users_mat[np.where(users == data.iloc[i][0])[0][0]]
            self.labels[i] = data.iloc[i][2]
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx], self.labels[idx]

In [ ]:
train_train_data = TagsMatDataset(train_data[:5000])
train_test_data = TagsMatDataset(train_data[5000:6000])

In [ ]:
trainloader = torch.utils.data.DataLoader(train_train_data, batch_size=5, shuffle=True)

print(next(iter(trainloader)))

In [ ]:
#tm_da, tm_la = next(iter(trainloader))
#print(tm_da.shape)

In [ ]:
epoch = 5
train_loss = 0
for eps in range(epoch):
    print('training', eps , 'epoch')
    for i, data in enumerate(trainloader, 0):
        model.train();
        inputs, labels = data
        labels = torch.tensor(labels, dtype=torch.long)
        optim.zero_grad()
#        input_user = users_mat[np.where(users == train_train_data.iloc[i][0])[0][0]]
#        input_job = jobs_mat[np.where(jobs == train_train_data.iloc[i][1])[0][0]]                    
#        input_data = torch.tensor([users_mat[np.where(users == train_train_data.iloc[i][0])[0][0]]])
#        label = torch.zeros(2)    # 0 -> [1,0], 1 -> [0, 1]
#        label[train_train_data.iloc[i][2]] = 1
#        label = train_train_data.iloc[i][2]
        answers = model(inputs.float())
        loss = criterion(answers, labels)
        loss.backward();
        optim.step()
        train_loss += loss.item()
        if i % 500 == 499:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, train_loss / 2000))
            train_loss = 0.0

In [ ]:
testloader = torch.utils.data.DataLoader(train_test_data, batch_size=5, shuffle=True)

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy: %d %%' % (
    100 * correct / total))

In [ ]:
print(train_data.groupby('userID').sum().applied)

user, job을 각각 임베딩한 다음 둘 사이의 유사도 계산

In [ ]:
class TagsMatDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        super().__init__()
        self.samples = np.ndarray(shape=(len(data), 2, len(tags)))    # user, job에 대한 태그보유여부가 0, 1로 표시된 행렬
        self.labels = np.zeros(shape=(len(data)))
        for i in range(len(data)):
            self.samples[i][0] = users_mat[np.where(users == data.iloc[i][0])[0][0]]
            self.samples[i][1] = jobs_mat[np.where(jobs == data.iloc[i][1])[0][0]]
            self.labels[i] = data.iloc[i][2]
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx][0], self.samples[idx][0], self.labels[idx]

In [ ]:
train_train_data = TagsMatDataset(train_data[:5000])
train_test_data = TagsMatDataset(train_data[5000:6000])

In [ ]:
trainloader = torch.utils.data.DataLoader(train_train_data, batch_size=10, shuffle=True)
testloader = torch.utils.data.DataLoader(train_test_data, batch_size=10, shuffle=True)
print(next(iter(trainloader)))

In [ ]:
#def cosineSim(x, y):
#    return cossim(x, y)[0][1]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, n_tags=887, d_embed=400, d_hidden=200, n_feature=100, dp=0.01):  #887차원의 데이터를 300차원으로 임베딩, 256개 은닉층을 거쳐 10개 데이터 아웃
        super().__init__()
        self.module1 = nn.Sequential(    # usertags embedding
        nn.Linear(n_tags, d_embed),
        nn.ELU(),
        nn.Dropout(dp),
        nn.BatchNorm1d(d_embed),
        nn.Linear(d_embed, d_hidden),
        nn.ELU(),
        nn.Dropout(dp),
        nn.BatchNorm1d(d_hidden),
        nn.Linear(d_hidden, n_feature),
        nn.ELU(),
        nn.BatchNorm1d(n_feature)
        )
        
        self.module2 = nn.Sequential(    # jobtags embedding
        nn.Linear(n_tags, d_embed),
        nn.ELU(),
        nn.Dropout(dp),
        nn.BatchNorm1d(d_embed),
        nn.Linear(d_embed, d_hidden),
        nn.ELU(),
        nn.Dropout(dp),
        nn.BatchNorm1d(d_hidden),
        nn.Linear(d_hidden, n_feature),
        nn.ELU(),
        nn.BatchNorm1d(n_feature)
        )
        self.ncfLayer = nn.Sequential(
#            nn.Linear(n_feature*2, n_feature),
#            nn.ELU(),
#            nn.Dropout(dp),
#            nn.BatchNorm1d(n_feature),
            nn.Linear(n_feature, 50),
            nn.ELU(),
            nn.Dropout(dp),
            nn.BatchNorm1d(50),
            nn.Linear(50, 2)
        )
#        self.fc1 = nn.Linear(n_feature*2, n_feature)
#        self.fc2 = nn.Linear(n_feature, 50)
#        self.fc3 = nn.Linear(50, 2)
#        self.cosineSim = lambda x, y : cossim(x, y)[0][1]
#        self.cossim = nn.CosineSimilarity(dim=1, eps=1e-6)
#        self.bias = nn.Parameter(torch.zeros(1))
    def forward(self, userTags, jobTags):
#        userTags = x[][0]
#        jobTags = x[1]
        userTags = self.module1(userTags)
        jobTags = self.module2(jobTags)
#        x = torch.cat((userTags, jobTags), dim=1)
#        x = self.ncfLayer(x)
        x = torch.mul(userTags, jobTags)
        x = self.ncfLayer(x)
#        sim = self.cossim(userTags, jobTags)
#        x = [1-sim, sim]
#        with torch.no_grad():
#            x = torch.empty()
        return x

In [ ]:
criterion = nn.CrossEntropyLoss()
model = Net()
optim = torch.optim.Adam(model.parameters(), lr=0.006, weight_decay=1e-2)

In [ ]:
epoch = 4
train_loss = 0
all_losses = []
for eps in range(epoch):
    print('training', eps+1 , 'epoch')
    for i, data in enumerate(trainloader, 0):
        model.train()
        user, job, labels = data
        labels = torch.tensor(labels, dtype=torch.long)
        optim.zero_grad()
#        input_user = users_mat[np.where(users == train_train_data.iloc[i][0])[0][0]]
#        input_job = jobs_mat[np.where(jobs == train_train_data.iloc[i][1])[0][0]]                    
#        input_data = torch.tensor([users_mat[np.where(users == train_train_data.iloc[i][0])[0][0]]])
#        label = torch.zeros(2)    # 0 -> [1,0], 1 -> [0, 1]
#        label[train_train_data.iloc[i][2]] = 1
#        label = train_train_data.iloc[i][2]
        answers = model(user.float(), job.float())
        loss = criterion(answers, labels)
        loss.backward();
        optim.step()
        train_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, train_loss / 100))
            all_losses.append(train_loss)
            train_loss = 0.0


In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(len(all_losses)), all_losses)
plt.show()

In [ ]:
testloader = torch.utils.data.DataLoader(train_test_data, batch_size=5, shuffle=True)

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        user, job, labels = data
        labels = torch.tensor(labels, dtype=torch.long)
        outputs = model(user.float(), job.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy: %d %%' % (
    100 * correct / total))

데이터에 유저 id를 추가로 반영

In [ ]:
class TagsMatDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        super().__init__()
        self.uid = data.userID
        self.samples = np.ndarray(shape=(len(data), 2, len(tags)))    # user, job에 대한 태그보유여부가 0, 1로 표시된 행렬
        self.labels = np.zeros(shape=(len(data)))
        for i in range(len(data)):
            self.samples[i][0] = users_mat[np.where(users == data.iloc[i][0])[0][0]]
            self.samples[i][1] = jobs_mat[np.where(jobs == data.iloc[i][1])[0][0]]
            self.labels[i] = data.iloc[i][2]
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.uid[idx], self.samples[idx][0], self.samples[idx][0], self.labels[idx]

train_train_data = TagsMatDataset(train_data[:5000])
train_test_data = TagsMatDataset(train_data[5000:6000])
trainloader = torch.utils.data.DataLoader(train_train_data, batch_size=10, shuffle=True)
testloader = torch.utils.data.DataLoader(train_test_data, batch_size=10, shuffle=True)
print(next(iter(trainloader)))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, n_tags=887, d_embed=400, d_hidden=200, n_feature=100, dp=0.01):  #887차원의 데이터를 300차원으로 임베딩, 256개 은닉층을 거쳐 10개 데이터 아웃
        super().__init__()
        self.module1 = nn.Sequential(    # usertags embedding
        nn.Linear(n_tags, d_embed),
        nn.ELU(),
        nn.Dropout(dp),
        nn.BatchNorm1d(d_embed),
        nn.Linear(d_embed, d_hidden),
        nn.ELU(),
        nn.Dropout(dp),
        nn.BatchNorm1d(d_hidden),
        nn.Linear(d_hidden, n_feature),
        nn.ELU(),
        nn.BatchNorm1d(n_feature)
        )
        
        self.module2 = nn.Sequential(    # jobtags embedding
        nn.Linear(n_tags, d_embed),
        nn.ELU(),
        nn.Dropout(dp),
        nn.BatchNorm1d(d_embed),
        nn.Linear(d_embed, d_hidden),
        nn.ELU(),
        nn.Dropout(dp),
        nn.BatchNorm1d(d_hidden),
        nn.Linear(d_hidden, n_feature),
        nn.ELU(),
        nn.BatchNorm1d(n_feature)
        )
        self.fc1 = nn.Linear(n_feature*2, 2)
        self.cosineSim = lambda x, y : cossim(x, y)[0][1]
        self.cossim = nn.CosineSimilarity(dim=1, eps=1e-6)
#        self.bias = nn.Parameter(torch.zeros(1))
    def forward(self, userTags, jobTags):
#        userTags = x[][0]
#        jobTags = x[1]
        userTags = self.module1(userTags)
        jobTags = self.module1(jobTags)
        x = torch.cat((userTags, jobTags), dim=1)
        x = self.fc1(x)
#        sim = self.cossim(userTags, jobTags)
#        x = [1-sim, sim]
#        with torch.no_grad():
#            x = torch.empty()
        return x

유저에게 회사 추천

In [ ]:
target_user = users_mat[40]    # 40번 유저에게 추천되는 상위 3개 회사 표시
print(target_user.shape)

In [ ]:
# 가장 유사한 job 검색

with torch.no_grad():
    bestmatch = -1
    for i in range(len(jobs_mat)):
        target_job = jobs_mat[i]
        target_job = target_job.repeat(5)
        output = model(target_user, target_job)
        output = nn.Softmax(output)
        if bestmatch < output[1]:
            bestmatch = output[1]
            bestjob = i

deep averaging network 모델(단어의 순서를 고려하지 않고 단어의 임베딩을 평균을 취해 만든다)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class DAN(nn.Module):

    def __init__(self,
                 n_embed=10000,
                 d_embed=300,
                 d_hidden=256,
                 d_out=2,
                 dp=0.2,
                 embed_weight=None):
        super(DAN, self).__init__()

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        self.embed = nn.Embedding(n_embed, d_embed)

        if embed_weight is not None:
            # embed_weight = inputs.vocab.vectors
            self.embed.weight.data.copy_(embed_weight)
            self.embed.weight.requires_grad = False

        self.dropout1 = nn.Dropout(dp)
        self.bn1 = nn.BatchNorm1d(d_embed)
        self.fc1 = nn.Linear(d_embed, d_hidden)
        self.dropout2 = nn.Dropout(dp)
        self.bn2 = nn.BatchNorm1d(d_hidden)
        self.fc2 = nn.Linear(d_hidden, d_out)

    def forward(self, batch):
        text = batch.text
        label = batch.label

        x = self.embed(text)

        x = x.mean(dim=0)

        x = self.dropout1(x)
        x = self.bn1(x)
        x = self.fc1(x)
        x = self.dropout2(x)
        x = self.bn2(x)
        x = self.fc2(x)
        return x

In [ ]:
"""
for user in users:
    #print(user)
    usertagfilter = user_tag["userID"].isin([user])
    taglist = user_tag[usertagfilter].tagID.values    # 각 유저에 대한 태그 확인
    if user == "fe292163d06253b716e9a0099b42031d":    # 해당 유저는 151개 태그가 존재(중복 포함)
        print(taglist.size)


In [ ]:
"""

In [ ]:
#print(train_data[:1])
#print(user_tag)
#print(train_data.columns)
for user in range(0, len(train_data)):
    id = train_data.loc[user, "userID"]
    if id == "fe292163d06253b716e9a0099b42031d":
        print(id)
        usertagfilter = user_tag["userID"].isin([id])
        usertag = user_tag[usertagfilter].tagID.values
        print(usertag)
#        print(user_tag["userID"].isin([id]))
#    print(user['userID'])

xptmxm


In [ ]:
def dot(list_a, list_b):
    return sum(a*b for a, b in zip(list_a, list_b))
def jaccard(list_a, list_b):
    return dot(list_a, list_b) / (sum(a for a in list_a) + sum(b for b in list_b) - dot(list_a, list_b))
def norm(list_a):
    return sum(a*a for a in list_a) ** 0.5
def cossine(list_a, list_b):
    return dot(list_a, list_b) / (norm(list_a) * norm(list_b))

In [ ]:
a = torch.rand(5, 5)
b = torch.rand(5, 5)

costest = nn.CosineSimilarity(dim = 1)
print(a[1], b[1])
print(costest(a, b))
print(cossine(a[0], b[0]))

In [ ]:
print(train_data.userID)

ㅊㄴㅊㄴㅊ

In [ ]:
import torch
from engine import Engine
from utils import use_cuda


class GMF(torch.nn.Module):
    def __init__(self, config):
        super(GMF, self).__init__()
        self.num_users = users.size
        self.num_items = jobs.size
        self.latent_dim = 8

        self.embedding_user = torch.nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.latent_dim)
        self.embedding_item = torch.nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.latent_dim)

        self.affine_output = torch.nn.Linear(in_features=self.latent_dim, out_features=1)
        self.logistic = torch.nn.Sigmoid()

    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        element_product = torch.mul(user_embedding, item_embedding)
        logits = self.affine_output(element_product)
        rating = self.logistic(logits)
        return rating

    def init_weight(self):
        pass


class GMFEngine(Engine):
    """Engine for training & evaluating GMF model"""
    def __init__(self, config):
        self.model = GMF(config)
#        if config['use_cuda'] is True:
#            use_cuda(True, config['device_id'])
#            self.model.cuda()
#        super(GMFEngine, self).__init__(config)

In [ ]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        super().__init__()
        self.sample = np.ndarray(shape=(len(data), 3))
        for i in range(len(data)):
            self.sample[i][0] = 
            self.sample[i][1] = np.where(jobs == data.iloc[i][1])[0][0]
            self.sample[i][2] = data.iloc[i][2]
    def __len__(self):
        return len(self.sample)

    def __getitem__(self, i):
        return self.sample[i]